##Check song  profile to make sure song ids match song ids in MSD
'caue the track ids don't seem to carry over  
(They do, yay! Well, except for the known mismatches, probably. Or at least I'm going to assume there are no other mismatches until I discover otherwise)  
```
http://developer.echonest.com/api/v4/song/profile?api_key=API_KEY&format=json&id=SOEBCBI12AF72A154F
```

In [5]:
import sys
import requests
import time
import pickle
from pprint import pprint

In [2]:
def pickle_stuff(filename, data):
    ''' save file '''
    with open(filename, 'w') as picklefile:
        pickle.dump(data, picklefile)

def unpickle(filename):
    ''' open file '''
    with open(filename, 'r') as picklefile:
        old_data = pickle.load(picklefile)
    return old_data

In [7]:
with open("echonestapi1.txt", 'r') as ena:
    api_key = ena.read().strip()

In [8]:
rate_time = 0.501 #3.1

In [3]:
r = requests.get('http://developer.echonest.com/api/v4/track/profile?api_key='+api_key+'&id=musicbrainz:track:4bfce89c-b1e6-407f-91ba-e13a81441a7d')

In [4]:
rdata = r.json()
pprint(rdata)

{u'response': {u'status': {u'code': 0,
                           u'message': u'Success',
                           u'version': u'4.2'},
               u'track': {u'artist': u'Hammock',
                          u'catalog': u'musicbrainz',
                          u'foreign_id': u'musicbrainz:track:4bfce89c-b1e6-407f-91ba-e13a81441a7d',
                          u'foreign_ids': [u'musicbrainz:track:4bfce89c-b1e6-407f-91ba-e13a81441a7d'],
                          u'foreign_release_id': u'musicbrainz:release:9f2cecb3-ca9a-4d29-8bcd-a692f32e2f11',
                          u'foreign_release_ids': [u'musicbrainz:release:9f2cecb3-ca9a-4d29-8bcd-a692f32e2f11'],
                          u'id': u'TRRYMSJ13B7BF722BF',
                          u'release': u'More Dead Than Alive (Get Away From the Medicine)',
                          u'song_id': u'SOCGKJR12A58A79D30',
                          u'status': u'complete',
                          u'title': u'I Can Almost See You'}}}


In [5]:
pprint(rdata["response"]["status"]["message"])

u'Success'


In [8]:
pprint(rdata["response"]["track"])
pprint(rdata["response"]["track"]["song_id"])

{u'artist': u'Hammock',
 u'catalog': u'musicbrainz',
 u'foreign_id': u'musicbrainz:track:4bfce89c-b1e6-407f-91ba-e13a81441a7d',
 u'foreign_ids': [u'musicbrainz:track:4bfce89c-b1e6-407f-91ba-e13a81441a7d'],
 u'foreign_release_id': u'musicbrainz:release:9f2cecb3-ca9a-4d29-8bcd-a692f32e2f11',
 u'foreign_release_ids': [u'musicbrainz:release:9f2cecb3-ca9a-4d29-8bcd-a692f32e2f11'],
 u'id': u'TRRYMSJ13B7BF722BF',
 u'release': u'More Dead Than Alive (Get Away From the Medicine)',
 u'song_id': u'SOCGKJR12A58A79D30',
 u'status': u'complete',
 u'title': u'I Can Almost See You'}
u'SOCGKJR12A58A79D30'


In [92]:
mbid_songs = {}
skiplines = 8600

In [93]:
with open("mbid_songs.csv", 'r') as msf:
    for line in msf:
        line = line.strip()
        segs = line.split(",")
        key = segs[0]
        track = segs[1]
        mbid_songs[key] = track

In [94]:
print len(mbid_songs.keys())

1212834


In [95]:
linecount = 0
with open("mbids.txt", 'r') as mf:
    for s in xrange(skiplines):
        next(mf)
    t0 = time.time()
    for line in mf:
        linecount += 1
        print len(line.split())
        t1 = time.time()
        count = 0
        for mbid in line.split():
            if mbid in mbid_songs:
                #print "continuing"
                count += 1
                continue
            print count,
            count += 1
            moveon = False
            while not moveon:
                #print "requesting"
                time.sleep(rate_time)
                try:
                    enr = requests.get('http://developer.echonest.com/api/v4/track/profile?api_key='+api_key+
                                       '&id=musicbrainz:track:'+mbid)
                    enrj = enr.json()
                    status = enrj["response"]["status"]["message"]
                    if status == "Success":
                        #print "true"
                        if "song_id" in enrj["response"]["track"]:
                            #print "success" #"ids", mbid
                            mbid_songs[mbid] = enrj["response"]["track"]["song_id"]
                        moveon = True
                    elif status == "The Identifier specified does not exist":
                        #echo nest does not have all mbids mapped to tracks
                        #print status #mbid, status
                        mbid_songs[mbid] = ""
                        moveon = True
                    else:
                        print status
                        time.sleep(3)
                except requests.exceptions.ConnectionError as e:
                    print e,
                    time.sleep(3)
        t2 = time.time()
        if t2-t1 > 0.5:
            print "\nline seconds:", t2-t1,
            print "total seconds:", t2-t0, 
            print "total lines:", linecount     


1526
1204
2248
464
950
396
909
485
2837
1354
1430
1692
790
1426
1455
2232
892
785
2841
1132
2179
0
736
733
413
761
1083
1433
209
1792
370
468
1371
530
664
1
1341
961
2735
292
675
1023
828
1461
1331
911
1396
983
35
2059
1458
303
16
465
269
1229
1013
511
655
1047
1002
1201
1605
0
586
471
562
1096
1477
1686
1494
1059
983
2104
1104
793
1630
1259
922
214
1269
2186
1663 1673 1676 1685 1686 1698 1699 1700 1714 1727 1732 1734 1736 1742 1751 1756 1760 1763 1766 1793 1794 1795 1797 1800 1812 1813 1815 1818 1825 1828 1829 1832 1835 1838 1839 1840 1844 1847 1855 1858 1862 1867 1869 1886 1889 1890 1893 1895 1897 1907 1908 1910 1916 1924 1927 1935 1936 1938 1940 1942 1947 1950 1953 1957 1958 1973 1983 1984 1989 1992 1994 1995 1999 2001 2003 2004 2007 2023 2028 2030 2039 2041 2044 2054 2063 2077 2089 2104 2107 2126 2127 2135 2148 2153 2154 2155 2156 2157 2161 2163 2171 2174 2180 2183 2185 
line seconds: 121.853049994 total seconds: 121.904340029 total lines: 82
265
15 46 76 81 87 119 254 
line second

In [96]:
print len(mbid_songs.keys())

1226050


In [97]:
with open("mbid_songs.csv", 'w') as msf:
    for k, v in mbid_songs.iteritems():
        msf.write(k+","+v+"\n")

In [116]:
#print mbid_songs

In [10]:
#pickle_stuff("mbid_song_dict.pkl", mbid_songs)